## Super Bowl Squares Average Payout Analysis

Gian Favero | February 2nd, 2024

We look to Stathead.com's DriveFinder to estimate the probability of any scoring event (TD, FG, safety) occurring during a given drive. We assume each drive is independent of another (no accommodations for game script) and will handle the scenario of PATs as a double scoring drive should a TD be the result of a drive. Historical PAT data shows nearly 100% success on average over the past ten years, so the chance of incurring a PAT is equal to the chance of scoring a TD. 

We additionally look to Pro Football Reference to determine how many drives on average each team gets in a given NFL game. We multiply this number by two to get the total number of offensive drives per game

In [6]:
import numpy as np

buy_in = 2
payout = 0.05
pot = 100 * buy_in

# Stats taken from Stathead.com Drive Finder over 2013 - 2023 seasons
p_TD = 0.213
p_FG = 0.144
p_S = 0.002
no_score = 1 - p_TD - p_FG - p_S

drive_probabilities = [p_TD, p_FG, p_S, no_score]

# Stat taken from Pro Football Reference over 2013 - 2023 seasons
drives_per_game = 11.5 * 2

Using this information, we now have enough data to run Monte Carlo simulations to determine the number of times each square will be landed on in a given NFL game. We consider each drive independently (again, not accounting for game script) and consider that a PAT occurs after each TD at a 100% conversion rate.

In [7]:
simulations = 10000
square_board = np.zeros((10, 10))

for i in range(simulations):

    team_1_score = 0
    team_2_score = 0

    for j in range(int(drives_per_game)):
        # Reset score for each drive
        score = 0

        # Select drive outcome based on probabilities
        drive_outcome = np.random.choice([0, 1, 2, 3], p=drive_probabilities)

        # Add score based on drive outcome
        if drive_outcome == 0:      # TD
            score += 6
        elif drive_outcome == 1:    # FG
            score += 3
        elif drive_outcome == 2:    # Safety
            score += 2
        else:                       # No score
            score = 0

        # Add score to team 1 or team 2 based on drive number
        if j % 2 == 0:
            team_1_score += score
        else:
            team_2_score += score

        # Update board if score occurred
        if score > 0:
            square_board[team_1_score % 10][team_2_score % 10] += 1

            # If result was a touchdown, add the extra point
            if drive_outcome == 0:
                if j % 2 == 0:
                    team_1_score += 1
                else:
                    team_2_score += 1

                square_board[team_1_score % 10][team_2_score % 10] += 1

# Normalize the board
square_board = square_board / simulations

# Print the board
print("Times landed per square: ")
np.set_printoptions(suppress=True)
print(square_board)

# Print the sum of the board
print("\nSum of the number of landings: ")
print(np.sum(square_board))
landings = np.sum(square_board)

Times landed per square: 
[[0.569 0.126 0.047 0.571 0.3   0.026 0.504 0.671 0.046 0.136]
 [0.143 0.046 0.017 0.106 0.087 0.011 0.072 0.132 0.021 0.035]
 [0.058 0.017 0.004 0.037 0.033 0.004 0.018 0.048 0.007 0.009]
 [0.703 0.102 0.035 0.404 0.233 0.021 0.248 0.471 0.037 0.094]
 [0.344 0.083 0.028 0.246 0.174 0.018 0.162 0.293 0.034 0.073]
 [0.032 0.01  0.003 0.024 0.021 0.004 0.013 0.03  0.006 0.005]
 [0.644 0.069 0.018 0.231 0.15  0.014 0.11  0.306 0.028 0.051]
 [0.8   0.124 0.044 0.454 0.271 0.025 0.345 0.534 0.047 0.124]
 [0.056 0.019 0.007 0.044 0.037 0.006 0.027 0.054 0.011 0.014]
 [0.172 0.035 0.008 0.109 0.073 0.006 0.047 0.128 0.014 0.018]]

Sum of the number of landings: 
13.1319


This probability per square graphic is essentially telling us how many times an individual square is expected to be landed on per game. We can then get the expected earnings per square through the expected payout per score.

In [8]:
# Calculate the payout per square
payout_board = square_board * payout * pot
print("Payout per square: ")
print(payout_board)

Payout per square: 
[[5.687 1.257 0.474 5.713 2.997 0.258 5.043 6.709 0.465 1.358]
 [1.433 0.464 0.169 1.065 0.871 0.112 0.722 1.316 0.209 0.345]
 [0.576 0.168 0.043 0.374 0.327 0.044 0.182 0.48  0.07  0.085]
 [7.033 1.021 0.352 4.041 2.329 0.209 2.476 4.712 0.369 0.944]
 [3.444 0.834 0.279 2.46  1.745 0.183 1.624 2.93  0.341 0.732]
 [0.317 0.096 0.033 0.237 0.215 0.037 0.134 0.303 0.063 0.052]
 [6.44  0.694 0.182 2.311 1.501 0.139 1.098 3.061 0.277 0.506]
 [7.997 1.237 0.442 4.542 2.714 0.253 3.455 5.34  0.47  1.243]
 [0.561 0.192 0.066 0.439 0.37  0.065 0.273 0.54  0.111 0.14 ]
 [1.715 0.345 0.083 1.09  0.733 0.063 0.47  1.282 0.139 0.179]]


We can additionally estimate the expected grand payout per square. Considering the full pot and an average of 13 payouts per game, the average grand prize will be 

$\text{GP} = \text{Pot} - \text{Payout} \times 13$.

 We have the average number of landings per square in a given game, so if we divide this by 13 we will have the probability of each square being landed on for one individual score in that game (which could be the final score). We multiply this by the expected grand prize to approximate the expected grand prize per square.

In [9]:
# Add a board that represents winning the grand prize according to an average of 13 payouts per game
normalized_board = square_board / landings
grand_board = normalized_board * (pot - pot * payout * landings)

# Print the grand prize board with 3 decimal places
print("Grand prize board: ")
np.set_printoptions(precision=3)
print(grand_board)

print("\nSum of the grand prize board: ")
print(np.sum(grand_board))

Grand prize board: 
[[2.974 0.657 0.248 2.988 1.567 0.135 2.638 3.509 0.243 0.71 ]
 [0.749 0.243 0.088 0.557 0.456 0.059 0.378 0.688 0.109 0.18 ]
 [0.301 0.088 0.022 0.196 0.171 0.023 0.095 0.251 0.037 0.044]
 [3.678 0.534 0.184 2.113 1.218 0.109 1.295 2.464 0.193 0.494]
 [1.801 0.436 0.146 1.287 0.913 0.096 0.849 1.532 0.178 0.383]
 [0.166 0.05  0.017 0.124 0.112 0.019 0.07  0.158 0.033 0.027]
 [3.368 0.363 0.095 1.209 0.785 0.073 0.574 1.601 0.145 0.265]
 [4.183 0.647 0.231 2.376 1.419 0.132 1.807 2.793 0.246 0.65 ]
 [0.293 0.1   0.035 0.23  0.194 0.034 0.143 0.282 0.058 0.073]
 [0.897 0.18  0.043 0.57  0.383 0.033 0.246 0.67  0.073 0.094]]

Sum of the grand prize board: 
68.68100000000001


We can now introduce the league constraints to the problem. Each square costs $2 for a maximum of ten squares. Squares are assigned randomly according to a uniform distribution. Using this information, we can attempt to discern if buying a particular number of squares maximizes our chances of breaking even.

In [10]:
simulations = 50000

# Run a loop over each purchasable number of squares
for i in range(10):
    payout = 0
    # Run a Monte Carlo simulation over a number of trials
    for j in range(simulations):
        # Randomly select i + 1 squares from the board
        team_1_index = np.random.choice(10, i+1)
        team_2_index = np.random.choice(10, i+1)

        # Sum the expected payouts for each square
        for k in range(i+1):
            payout += payout_board[team_1_index[k]][team_2_index[k]] + grand_board[team_1_index[k]][team_2_index[k]]

    # Print the expected payout for each number of squares
    print(f"Expected payout for {i+1} squares: {payout/simulations}, Profit: {payout/simulations - 2 * (i+1)}")

Expected payout for 1 squares: 1.9984090649488093, Profit: -0.0015909350511906784
Expected payout for 2 squares: 3.9993738301391306, Profit: -0.0006261698608693855
Expected payout for 3 squares: 5.982606203215382, Profit: -0.017393796784618054
Expected payout for 4 squares: 8.004423548762473, Profit: 0.004423548762472862
Expected payout for 5 squares: 9.992858641932033, Profit: -0.0071413580679671185
Expected payout for 6 squares: 11.98320562903791, Profit: -0.016794370962090355
Expected payout for 7 squares: 14.027123538857783, Profit: 0.027123538857782847
Expected payout for 8 squares: 15.950785522270792, Profit: -0.049214477729208284
Expected payout for 9 squares: 17.997690539820308, Profit: -0.0023094601796920244
Expected payout for 10 squares: 19.992941432682198, Profit: -0.0070585673178023


We observe no bias towards those who buy as many squares as possible. Over time, it is expected that you will break even regardless of strategy.